In [1]:
!pip install pysurvival

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
  Created wheel for pysurvival: filename=pysurvival-0.1.2-cp37-cp37m-linux_x86_64.whl size=3773518 sha256=a01141eedcd41cab82efcdc5b1ab144becd2949c811398b8d72c38529e520eaf
  Stored in directory: /root/.cache/pip/wheels/1a/63/e2/32273d765a4e2f4ccac69c8adf97425ca80bab5d0c8447f120
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=790f1f2f08bab91977a7abfc700853d67d90f6fc1b96fd58743158f32f9f376e
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=8320202bc284df83455418bea4835632f4a89c6731665200ef6aa933323fd3b7
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pysurvival progressbar

# Original

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. train_data _73.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.817829


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_original.zip')

/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_original.zip


# TGAN-imbal-5000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.804285


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-5000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-5000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-10000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.805449


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-10000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-10000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-15000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.806463


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-15000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-15000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-20000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.805023


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-20000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-20000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-25000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.810506


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-25000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-25000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-30000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.806738


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-30000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-30000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-35000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.805956


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-35000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-35000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-40000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.803876


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-40000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-40000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-45000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.807271


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-45000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-45000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-imbal-50000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.805307


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-50000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-imbal-50000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-5000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.805201


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-5000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-5000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-10000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.806258


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-10000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-10000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-15000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.806711


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-15000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-15000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-20000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.806356


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-20000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-20000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-25000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

KeyboardInterrupt: ignored

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-25000.zip')

# TGAN-bal-30000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.807644


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-30000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-30000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-35000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.806418


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-35000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-35000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-40000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.804205


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-40000-2.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-40000-2.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-45000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.804952


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-45000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-45000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-bal-50000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.803370


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-50000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-bal-50000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-5000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820415


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-5000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-5000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-10000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.822183


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-10000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-10000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-15000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820148


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-15000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-15000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-20000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.818246


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-20000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-20000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-25000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820006


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-25000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-25000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-30000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.817082


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-30000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-30000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-35000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.819011


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-35000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-35000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-40000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820593


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-40000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-40000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-45000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.819375


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-45000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-45000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-aug-50000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820246


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-50000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-aug-50000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-6616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-6616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.811919


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-6616(2).zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-6616(2).zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-11616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-11616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.809200


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-11616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-11616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-16616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-16616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.808826


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-11616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-11616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-21616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-21616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.804374


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-21616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-21616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-26616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-26616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.807911


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-26616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-26616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-31616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-31616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.807360


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-31616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-31616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-36616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-36616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.808729


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-36616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-36616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-41616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-41616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.803441


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-41616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-41616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-46616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-46616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.804161


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-46616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-46616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGAN-org+balAR-51616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-51616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.805947


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-51616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+balAR-51616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-6616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-6616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.821215


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-6616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-6616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-11616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-11616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.821686


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-11616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-11616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-16166

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-16166.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820513


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-16166.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-16166.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-21616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-21616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
final_train_df = new_train_df
final_test_df = new_test_df

In [ ]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [ ]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.821241


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-21616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-21616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-26616

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-26616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [4]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [6]:
final_train_df = new_train_df
final_test_df = new_test_df

In [7]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [8]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [9]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [10]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [11]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820450


In [12]:
from pysurvival.utils import save_model

In [13]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-26616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-26616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-31616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-31616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
final_train_df = new_train_df
final_test_df = new_test_df

In [6]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [7]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [8]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [9]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [10]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.819313


In [11]:
from pysurvival.utils import save_model

In [12]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-31616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-31616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-36616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-36616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
final_train_df = new_train_df
final_test_df = new_test_df

In [6]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [7]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [8]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [9]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [10]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.819970


In [11]:
from pysurvival.utils import save_model

In [12]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-36616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-36616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-41616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-41616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
final_train_df = new_train_df
final_test_df = new_test_df

In [6]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [7]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [8]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [9]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [10]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.821312


In [11]:
from pysurvival.utils import save_model

In [12]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-41616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-41616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-46616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-46616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
final_train_df = new_train_df
final_test_df = new_test_df

In [6]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [7]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [8]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [9]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [10]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.819571


In [11]:
from pysurvival.utils import save_model

In [12]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-46616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-46616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# TGANaug+aug-51616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.survival_forest import RandomSurvivalForestModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-51616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
final_train_df = new_train_df
final_test_df = new_test_df

In [6]:
features = [f for f in final_train_df.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [7]:
X_train, X_test = final_train_df[features], final_test_df[features]
T_train, T_test = final_train_df['a_year_time'].values, final_test_df['a_year_time'].values
E_train, E_test = final_train_df['a_year_event'].values, final_test_df['a_year_event'].values

In [8]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [9]:
rsf = RandomSurvivalForestModel(num_trees=200)
rsf.fit(X_train, T_train, E_train,
        max_features="sqrt", max_depth=5, min_node_size=20)

RandomSurvivalForestModel

In [10]:
c_index = concordance_index(rsf, X_test, T_test, E_test) #0.81
print('C-index: {:f}'.format(c_index))

C-index: 0.820530


In [11]:
from pysurvival.utils import save_model

In [12]:
save_model(rsf,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-51616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/Random survival forest/RSF_new(onehot)_tgan-org+imbalnonAR-51616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)
